# Setup Environment
Configure the Google Colab environment for working with geospatial data.

In [ ]:
# Setup Environment

# Install necessary packages
!pip install geopandas
!pip install pandas

# Import required libraries
import os
import pandas as pd
import geopandas as gpd

# Verify installations
print("Geopandas version:", gpd.__version__)
print("Pandas version:", pd.__version__)

# Download the NSI data
Need to download the NSI data from the site.

In [ ]:
# Install required packages if needed
!pip install requests tqdm

import requests
import os
import zipfile
from tqdm.notebook import tqdm

def download_and_extract_zip(url, target_dir="."):
    """
    Download a zip file from a URL and extract its contents.
    
    Args:
        url (str): URL of the zip file to download
        target_dir (str): Directory to extract the contents to
    """
    # Create target directory if it doesn't exist
    os.makedirs(target_dir, exist_ok=True)
    
    # Get the filename from URL
    filename = url.split('/')[-1]
    filepath = os.path.join(target_dir, filename)
    
    # Download the file with progress bar
    print(f"Downloading {filename}...")
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(filepath, 'wb') as f, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            bar.update(size)
    
    # Extract the zip file
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filepath, 'r') as zip_ref:
        zip_ref.extractall(target_dir)
    
    print(f"Download and extraction complete. Files extracted to {target_dir}")
    
    # Return the path to the extracted gpkg file (assumes there's only one gpkg file)
    extracted_files = [f for f in os.listdir(target_dir) if f.endswith('.gpkg')]
    if extracted_files:
        return os.path.join(target_dir, extracted_files[0])
    return None

# Download and extract the NSI data
nsi_url = "https://nsi.sec.usace.army.mil/downloads/nsi_2022/nsi_2022_15.gpkg.zip"
gpkg_path = download_and_extract_zip(nsi_url, "nsi_data")

print(f"GeoPackage path: {gpkg_path}")

# Run the Analysis
Implement the NsiBuildings class that handles loading and processing NSI building data from GeoPackage files.

In [ ]:
import os
from fortis.engine.analyses.hazus_flood import HazusFloodAnalysis
from fortis.engine.models.flood_depth_grid import FloodDepthGrid
from fortis.engine.vulnerability.default_flood import DefaultFloodFunction
from fortis.engine.models.nsi_buildings import NsiBuildings

# Define file paths (adjust these paths as necessary)
base_dir = os.path.dirname(__file__)
buildings_gpkg = os.path.join(base_dir, "nsi_2022_15.gpkg")
tif_file = os.path.join(base_dir, "Oahu_10_withReef.tif")

# Load NSI buildings data from GPKG
buildings = NsiBuildings(buildings_gpkg, "nsi")

# Read the depth grid from the TIFF file
depth_grid = FloodDepthGrid(tif_file)

# Create an instance of the default flood function
flood_function = DefaultFloodFunction(buildings, flood_type="R")

# Create the Hazus flood analyzer instance
analyzer = HazusFloodAnalysis(
    buildings=buildings,
    vulnerability_func=flood_function,
    depth_grid=depth_grid,
)

# Calculate losses using the analysis
analyzer.calculate_losses()

# Save the results to multiple formats
results_base = os.path.join(base_dir, "nsi_flood_losses")

# 1. Save as CSV
csv_path = f"{results_base}.csv"
buildings.gdf.to_csv(csv_path, index=False)

# 2. Save as GeoPackage
gpkg_path = f"{results_base}.gpkg"
buildings.gdf.to_file(gpkg_path, driver="GPKG")

print(f"NSI Flood analysis of {len(buildings.gdf):,} buildings complete.")
print(f"Results saved to:")
print(f"  - CSV: {csv_path}")
print(f"  - GeoPackage: {gpkg_path}")

: 

# Visualize Building Data
Create visualizations of the NSI building data including maps of building locations and foundation types.

In [ ]:
# Visualize Building Data

# Import necessary libraries for visualization
import matplotlib.pyplot as plt
import contextily as ctx

# Plot the building locations
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, marker='o', color='red', markersize=5, alpha=0.5)
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.Stamen.TonerLite)
ax.set_title("Building Locations")
plt.show()

# Plot the foundation types
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(column='foundation_type', ax=ax, legend=True, legend_kwds={'bbox_to_anchor': (1, 1)})
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.Stamen.TonerLite)
ax.set_title("Building Foundation Types")
plt.show()